전체 문서 모든 단어에 토픽이 이미 할당되어있다고 가정하였을떄, 아래와 같이 두 개의 형렬을 만들 수 있다.

**1. 단어-토픽 행렬**

    단어가 어떤 토픽에 할당됬는지 행렬로 구성.


**2. 문서-토픽 행렬**

    문서가 어떤 토픽에 할당됬는지 행렬로 구성

그렇다면 문서 내 임의의 단어에서 잠재된 토픽은 어떻게 추론할 수 있을까?

**'깁스 샘플링'** 기법을 이용하여, 추론한다. 

아래의 수식은 예제로 이해해보자.

${ \alpha  }, { \beta }$ 는 하이퍼 파라미터 이다.

$$p({ z }_{d, i }=j|{ z }_{ -i },w)=\frac { { n }_{ d,k }+{ \alpha  }_{ j } }{ \sum _{ i=1 }^{ K }{ ({ n }_{ d,i }+{ \alpha  }_{ i }) }  } \times \frac { { v }_{ k,{ w }_{ d,n } }+{ \beta  }_{ { w }_{ d,n } } }{ \sum _{ j=1 }^{ V }{ ({ v }_{ k,j }+{ \beta  }_{ j }) }  }=AB$$

$w_{(1,n)}$ 을 단어 $n$개로 구성된, 문서 1 그리고 토픽 개수는 3개라고 가정해보자.

$n$ 중에서 $i$번쨰 단어의 토픽을 추론한다고 예를 들어보자.

먼저 위의 모든 단어에 토픽이 이미 할당되어있다고 가정하였기 때문에, $i$번쨰 단어의 토픽을 지운다.

그렇다면, 문서1의 토픽 분포에 영향을 받는 값이 바로 $A$인데,

위의 수식에서 $n_{d,i}$가 뜻하는건, $i$는 $K$개의 토픽까지 이동하며, 문서$d$의 토픽 분포를 나타낸다.

$i$번쨰 단어의 토픽이 2였다고 하였을때, 제거한된 후 문서1의 토픽 분포이다.

<img src='http://i.imgur.com/zItjMJ9.png'>

토픽이 제거되었기 떄문에, 위의 **`단어-토픽 행렬`** 도 업데이트가 된다.

수식에서 B값이 영향을 받는다.

$v_{k,j}$는 단어 $j$를 $V$까지 이동시키면서 토픽 $k$에서 출현할 빈도 수이다.

<img src='http://i.imgur.com/c4TaEw3.png'>

위에서 나온 $A,B$를 직사각형의 높이와 너비로 둔다면, 문서 내 임의의 단어를 추론할 떄는

어떤 토픽이 될 확률은 직사각형의 넓이로 나타나게 됩니다.

<img src='http://i.imgur.com/AzeeBUd.png'>

In [2]:
from konlpy.tag import Okt, Komoran, Hannanum, Kkma
from eunjeon import Mecab

def get_tokenizer(tokenizer_name):

    tokenizer_dict={
        'komoran':Komoran(),
        'okt':Okt(),
        'mecab':Mecab(),
        'hannanum':Hannanum(),
        'kkma':Kkma()
    }
    try:
        tokenizer=tokenizer_dict[tokenizer_name]
    except:
        tokenizer=Mecab()
    return tokenizer

In [4]:
from gensim import corpora

corpus_fname = '../data/processed/corrected_ratings_corpus.txt'

documents, tokenized_corpus = [], []
tokenizer = get_tokenizer('mecab')

with open(corpus_fname, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(tokens)

In [5]:
dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [10]:
tokens

['포', '영차', '영차영차', '풍', '가', '신다', '나가', '저그']

In [22]:
document[:-1]

'포풍저그가나가신다영차영차영차'

In [14]:
tokenized_corpus[-1]

['포', '영차', '영차영차', '풍', '가', '신다', '나가', '저그']

In [32]:
dictionary.doc2bow(tokenized_corpus[-1])

[(12, 1),
 (906, 1),
 (2324, 1),
 (4617, 1),
 (12911, 1),
 (39005, 1),
 (60973, 1),
 (60974, 1)]

In [35]:
dictionary[60974]

'영차영차'

In [24]:
from gensim.models import ldamulticore

LDA = ldamulticore.LdaMulticore(corpus, id2word=dictionary, num_topics=30, workers=6)
all_topics = LDA.get_document_topics(corpus, minimum_probability=0.5, per_word_topics=False)


for doc_idx, topic in enumerate(all_topics[:5]):
    print(doc_idx, topic)

0 []
1 [(2, 0.9817264)]
2 [(7, 0.9431222)]
3 [(15, 0.58455116)]
4 [(25, 0.9306718)]


In [38]:
output_fname='../data/sentence-embeddings/lda/lda'

with open(output_fname + ".results", 'w', encoding='utf8') as f:
    for doc_idx, topic in enumerate(all_topics):
        if len(topic) == 1:
            topic_id, prob = topic[0]
            f.writelines(documents[doc_idx].strip() + "\u241E" + ' '.join(tokenized_corpus[doc_idx]) + "\u241E" + str(topic_id) + "\u241E" + str(prob) + "\n")
LDA.save(output_fname + ".model")

In [43]:
from collections import defaultdict
from gensim.models import Doc2Vec, LdaModel

class LDAEvaluator:

    def __init__(self, model_path="data/lda.results", tokenizer_name="mecab"):
        self.tokenizer = get_tokenizer(tokenizer_name)
        self.all_topics = self.load_results(model_path + ".results")
        self.model = LdaModel.load(model_path + ".model")

    def load_results(self, results_fname):
        topic_dict = defaultdict(list)
        with open(results_fname, 'r', encoding='utf-8') as f:
            for line in f:
                sentence, _, topic_id, prob = line.strip().split("\u241E")
                topic_dict[int(topic_id)].append((sentence, float(prob)))
        for key in topic_dict.keys():
            topic_dict[key] = sorted(topic_dict[key], key=lambda x: x[1], reverse=True)
        return topic_dict

    def show_topic_docs(self, topic_id, topn=10):
        return self.all_topics[topic_id][:topn]

    def show_topic_words(self, topic_id, topn=10):
        return self.model.show_topic(topic_id, topn=topn)

    def show_new_document_topic(self, documents):
        tokenized_documents = [self.tokenizer.morphs(document) for document in documents]
        curr_corpus = [self.model.id2word.doc2bow(tokenized_document) for tokenized_document in tokenized_documents]
        topics = self.model.get_document_topics(curr_corpus, minimum_probability=0.5, per_word_topics=False)
        for doc_idx, topic in enumerate(topics):
            if len(topic) == 1:
                topic_id, prob = topic[0]
                print(documents[doc_idx], ", topic id:", str(topic_id), ", prob:", str(prob))
            else:
                print(documents[doc_idx], ", there is no dominant topic")

In [44]:
model= LDAEvaluator(output_fname)

In [45]:
model.show_topic_docs(topic_id=10)

[('같이 성장한 나루토를 보며 감동을 느끼고 아쉬움을 느낀 작품이였다. ost도 아주 좋고 나루토를 위해 자신의 목숨까지 바치는 히나타의 마음을 이해한 나루토 그리고 한층 더 성장한 친구들내 기준에서는 최고의 명작이다!',
  0.9780216),
 ('오랜만에 마음이 따뜻하고 왠지 가슴저미는 이쁜 사랑이야기를 보게 되어 좋네요! 강동하, 이봄이 캐릭터 너무 사랑스럽고 감우성과 최수영씨 연기 너무 좋아 잘보고 있습니다!',
  0.97519594),
 ('오래 기다렸던 장률 감독의 첫 다큐멘터리 풍경. 역시나 명불허전. 눈오는 겨울에 딱 어울리는 영화네요. 영화가 끝난 후에도 요즈음 내 꿈은, 나는 어땠는지 계속 돌이켜보게 됩니다.',
  0.9751848),
 ('등장인물 몰입도가 상당합니다 배역을 어쩜 저리 잘맞았는지 감탄! 연기도 잘하고 미워도 미워할수 없는 케릭터들 16회로 끝나는 줄 알골 깜놀했다가 안심!',
  0.97516954),
 ('おもひでぽろぽろ 솔직하고 유머스러운 연출력이 돋보인 작품. 시골에 대한 정겨운 추억과 애틋한 감정이 남아 있다면 소소한 미소그를 머금으며 여유롭게 즐길 수 있는 영화~~',
  0.97511625),
 ('솔직히 내 취향에 직격. 2014년 최고의 영화였다. 여배우도 아니고 우둘투둘한 괴수의 전신노출을 보기 위해 한 시간 넘게 침을 꼴깍 삼키는 참신한 경험을 해 보았다.',
  0.9745215),
 ('역시 믿고보는 제이크 질렌할! LA경찰청은 감독한테 상줘야함. 두번 줘야함. 그들의 24시간 365일이 궁금하다. 계속 보고 싶다. 정규 시리즈제작 요망',
  0.97384953),
 ('잔잔한 웃음과 함께 상황을 재미있게 끌어가는 방송..너무 재미있게 보고있어요..디지털 시대에 아날로그 방송으로 사람의 마음을 뺏네요^^♡',
  0.9677702),
 ('비록 요즘 영화처럼 자극적인 과장은 없지만 황신혜를 향한 안성기의 진심어린 연기에 동정이 가는 배창호감독의 작품성이 엿보이는 영화',
  0.9677588)

In [47]:
model.show_topic_words(topic_id=10)

[('는', 0.03412451),
 ('.', 0.029883359),
 ('영화', 0.029247036),
 ('하', 0.024107441),
 ('다', 0.019369822),
 ('이', 0.018683344),
 ('을', 0.018298717),
 ('에', 0.015400472),
 ('게', 0.015370754),
 ('한', 0.0152194975)]

In [46]:
model.show_new_document_topic(['너무 사랑스러운 영화', '인생을 말하는 영화'])

너무 사랑스러운 영화 , topic id: 3 , prob: 0.8066544
인생을 말하는 영화 , topic id: 29 , prob: 0.62147003
